# Vietnamese ULMFiT from scratch

In [250]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import torch
from pathlib import Path
import os
import nlputils

from fastai import *
from fastai.text.all import *

In [251]:
# bs=48
# bs=24
bs=128

In [252]:
torch.cuda.set_device(0)

In [262]:
data_path = Path.cwd()

NameError: name 'ls' is not defined

This will create a `viwiki` folder, containing a `viwiki` text file with the wikipedia contents. (For other languages, replace `vi` with the appropriate code from the [list of wikipedias](https://meta.wikimedia.org/wiki/List_of_Wikipedias).)

In [254]:
lang = 'vi'
# lang = 'zh'

In [261]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

## Vietnamese wikipedia model

### Download data

In [264]:
from nlputils import split_wiki,get_wiki

In [263]:
get_wiki(path,lang)

downloading...


NameError: name 'download_url' is not defined

In [121]:
path.ls()

(#4) [Path('/home/marco/dlStems/FastAI/viwiki/wikiextractor'),Path('/home/marco/dlStems/FastAI/viwiki/viwiki-latest-pages-articles.xml.bz2'),Path('/home/marco/dlStems/FastAI/viwiki/viwiki-latest-pages-articles.xml'),Path('/home/marco/dlStems/FastAI/viwiki/viwiki')]

In [122]:
!head -n4 {path}/{name}

<doc id="13" url="https://vi.wikipedia.org/wiki?curid=13" title="Tiếng Việt">
Tiếng Việt

Tiếng Việt, cũng gọi là tiếng Việt Nam hay Việt ngữ là ngôn ngữ của người Việt và là ngôn ngữ chính thức tại Việt Nam. Đây là tiếng mẹ đẻ của khoảng 85% dân cư Việt Nam cùng với hơn 4 triệu Việt kiều. Tiếng Việt còn là ngôn ngữ thứ hai của các dân tộc thiểu số tại Việt Nam và là ngôn ngữ dân tộc thiểu số được công nhận tại Cộng hòa Séc.


This function splits the single wikipedia file into a separate file per article. This is often easier to work with.

In [227]:
dest = split_wiki(path,lang)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000


In [232]:
dest.ls()

(#87796) [Path('/home/marco/dlStems/FastAI/viwiki/docs/20490.txt'),Path('/home/marco/dlStems/FastAI/viwiki/docs/7241.txt'),Path('/home/marco/dlStems/FastAI/viwiki/docs/35276.txt'),Path('/home/marco/dlStems/FastAI/viwiki/docs/48904.txt'),Path('/home/marco/dlStems/FastAI/viwiki/docs/50873.txt'),Path('/home/marco/dlStems/FastAI/viwiki/docs/73476.txt'),Path('/home/marco/dlStems/FastAI/viwiki/docs/67988.txt'),Path('/home/marco/dlStems/FastAI/viwiki/docs/68587.txt'),Path('/home/marco/dlStems/FastAI/viwiki/docs/11623.txt'),Path('/home/marco/dlStems/FastAI/viwiki/docs/84915.txt')...]

In [229]:
dest

Path('/home/marco/dlStems/FastAI/viwiki/docs')

In [230]:
for count, filename in enumerate(os.listdir(a)):
    name = str(count)
    src = f"{dest}/{filename}"
    dst = f"{dest}/{count}.txt"
    os.rename(src, dst)

In [245]:
data = TextDataLoaders.from_folder(dest, valid_pct = 0.1, bs=1, verbose=True)

Found 87796 items
2 datasets of sizes 79017,8779
Setting up Pipeline: Tokenizer -> Numericalize
Setting up Pipeline: parent_label -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
Setting up after_item: Pipeline: ToTensor
Setting up before_batch: Pipeline: Pad_Chunk -- {'pad_idx': 1, 'pad_first': True, 'seq_len': 72}
Setting up after_batch: Pipeline: 


In [246]:
data.show_batch()

,text,category
0,"xxbos xxmaj chiến tranh đông xxmaj dương \n\n xxmaj chiến tranh đông xxmaj dương , còn được gọi là xxmaj chiến tranh đông xxmaj dương lần thứ nhất là một cuộc xung đột diễn ra tại ba nước đông xxmaj dương bao gồm xxmaj việt xxmaj nam , xxmaj lào và xxmaj campuchia , giữa một bên là xxmaj quân đội xxmaj viễn chinh xxmaj pháp cùng các lực lượng đồng minh bản xứ bao gồm lực lượng của xxmaj quốc gia xxmaj việt xxmaj nam , xxmaj vương quốc xxmaj lào , xxmaj vương quốc xxmaj campuchia thuộc xxmaj liên hiệp xxmaj pháp , bên kia là lực lượng xxmaj việt xxmaj nam xxmaj dân chủ xxmaj cộng hòa ( việt xxmaj minh ) cùng các lực lượng kháng chiến khác của xxmaj lào ( pathet xxmaj lào ) và xxmaj campuchia ( khmer đỏ ) . xxmaj cuộc",docs


In [125]:
# Use this to convert Chinese traditional to simplified characters
# ls *.txt | parallel -I% opencc -i % -o ../zhsdocs/% -c t2s.json

### Create pretrained model

In [235]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

NameError: name 'TextList' is not defined

In [ ]:
data = load_data(path, f'{lang}_databunch', bs=bs)

In [247]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [248]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [249]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7, 0.8))

epoch,train_loss,valid_loss,time


RuntimeError: CUDA out of memory. Tried to allocate 812.00 MiB (GPU 0; 3.82 GiB total capacity; 2.13 GiB already allocated; 206.38 MiB free; 2.20 GiB reserved in total by PyTorch)

Save the pretrained model and vocab:

In [ ]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

## Vietnamese sentiment analysis

### Language model

- [Data](https://github.com/ngxbac/aivivn_phanloaisacthaibinhluan/tree/master/data)
- [Competition details](https://www.aivivn.com/contests/1)
- Top 3 f1 scores: 0.900, 0.897, 0.897

In [35]:
train_df = pd.read_csv(path/'train.csv')
train_df.loc[pd.isna(train_df.comment),'comment']='NA'
train_df.head()

,id,comment,label
0,train_000000,Dung dc sp tot cam on \nshop Đóng gói sản phẩm...,0
1,train_000001,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng...,0
2,train_000002,Chất lượng sản phẩm tuyệt vời nhưng k có hộp ...,0
3,train_000003,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọ...,1
4,train_000004,Lần trước mình mua áo gió màu hồng rất ok mà đ...,1


In [36]:
test_df = pd.read_csv(path/'test.csv')
test_df.loc[pd.isna(test_df.comment),'comment']='NA'
test_df.head()

,id,comment
0,test_000000,Chưa dùng thử nên chưa biết
1,test_000001,Không đáng tiềnVì ngay đợt sale nên mới mua n...
2,test_000002,Cám ơn shop. Đóng gói sản phẩm rất đẹp và chắc...
3,test_000003,Vải đẹp.phom oki luôn.quá ưng
4,test_000004,Chuẩn hàng đóng gói đẹp


In [37]:
df = pd.concat([train_df,test_df], sort=False)

In [38]:
data_lm = (TextList.from_df(df, path, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [17]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [39]:
lr = 1e-3
lr *= bs/48

In [19]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.975080,4.138585,0.317773,00:07
1,4.408635,4.025489,0.326423,00:07


In [20]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.142114,3.928278,0.336230,00:09
1,4.010835,3.793583,0.349972,00:09
2,3.873617,3.694702,0.357240,00:09
3,3.761377,3.632186,0.364648,00:09
4,3.679017,3.595601,0.366964,00:09
5,3.614548,3.576386,0.369224,00:09
6,3.575895,3.567496,0.370285,00:09
7,3.560278,3.566525,0.370173,00:10


In [21]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [40]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='label')
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [10]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)

In [41]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [43]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [44]:
lr=2e-2
lr *= bs/48

In [45]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,_inner,time
0,0.338150,0.275298,0.899876,0.878430,00:02
1,0.302302,0.245949,0.902985,0.877226,00:02


In [46]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,_inner,time
0,0.321768,0.255457,0.899254,0.871367,00:02
1,0.305934,0.250888,0.894901,0.872021,00:02


In [47]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,_inner,time
0,0.300939,0.261080,0.893657,0.866201,00:03
1,0.263790,0.220207,0.906716,0.886115,00:03


In [48]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,_inner,time
0,0.282888,0.238203,0.905473,0.886483,00:04
1,0.248599,0.216489,0.918532,0.901550,00:04


In [49]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,_inner,time
0,0.201508,0.217176,0.911070,0.890084,00:05


In [50]:
learn_c.save(f'{lang}clas')

Competition top 3 f1 scores: 0.90, 0.89, 0.89. Winner used an ensemble of 4 models: TextCNN, VDCNN, HARNN, and SARNN.

## Ensemble

In [65]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load(f'{lang}clas', purge=False);

In [69]:
preds,targs = learn_c.get_preds(ordered=True)
accuracy(preds,targs),f1(preds,targs)

(tensor(0.9111), tensor(0.8952))

In [67]:
data_clas_bwd = load_data(path, f'{lang}_textlist_class_bwd', bs=bs, num_workers=1, backwards=True)
learn_c_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c_bwd.load(f'{lang}clas_bwd', purge=False);

In [70]:
preds_b,targs_b = learn_c_bwd.get_preds(ordered=True)
accuracy(preds_b,targs_b),f1(preds_b,targs_b)

(tensor(0.9092), tensor(0.8957))

In [71]:
preds_avg = (preds+preds_b)/2

In [72]:
accuracy(preds_avg,targs_b),f1(preds_avg,targs_b)

(tensor(0.9154), tensor(0.9016))